## Dual numbers and automatic differentiation
Dual numbers are defined as an algebraic extension over the field of real numbers by the nilpotent element __d__ such that **d**^2 = 0

The minimal polynomial of **d** being X^2, this extension is of degree 2 and every element in the base (1,__d__) follows the sum and multiplication rules :
* (x0,y0) + (x1,y1) = (x0+x1,y0+y1)
* (x0,y0) * (x1,y1) = (x0*x1,x0*y1+x1*y0)

Interestingly, when applying a polynomial P(X) = a0 + a1*X + a2*X^2 +... to a dual number, the following identity holds : 
* P((x,y)) = (P(x),P'(x)y)
where P'(X) = a1 + 2*a2*X+... is the derivative of P

In [1]:
import numpy as np
from numpy import random as rd

In [2]:
class Dual:
    def __init__(self, x: float, y: float) -> None:
        self._x = x
        self._y = y
    
    def __str__(self) -> str: 
        template = '({0._x}, {0._y})'
        return template.format(self)
    
    def __eq__(self, rhs) -> bool:
        return self._x == rhs._x and self._y == rhs._y
    
    def __ne__(self, rhs) -> bool:
        return not self.__eq__(rhs)
    
    def __add__(self, rhs: 'Dual') -> 'Dual':
        # if rhs is a scalar, cast it into a dual
        if type(rhs).__name__ != 'Dual':
            rhs = Dual(rhs, 0)
        return Dual(self._x+rhs._x, self._y+rhs._y)
    
    def __sub__(self, rhs: 'Dual') -> 'Dual':
        # if rhs is a scalar, cast it into a dual
        if type(rhs).__name__!= 'Dual':
            rhs = Dual(rhs, 0)
        return Dual(self._x-rhs._x, self._y-rhs._y)
    
    def __mul__(self, rhs: 'Dual') -> 'Dual':
        # if rhs is a scalar, cast it into a dual
        if type(rhs).__name__ != 'Dual':
            rhs = Dual(rhs, 0)
        return Dual(self._x * rhs._x, self._x * rhs._y+rhs._x * self._y)
        
    def __pow__(self, rhs: 'int' ) -> 'Dual':
        """Fast exponentiation : square and multiply"""
        if rhs == 0:
            return Dual(1,0)
        elif rhs % 2 == 0: 
            return self ** (rhs/2) * self ** (rhs/2);
        else:
            return self * ( self ** (rhs-1) );
    
    def norm(self) -> 'float':
        """Euclidean 2D norm"""
        return np.sqrt(self._x ** 2 + self._y **2)

In [3]:
class Polynomial:
    def __init__(self, coeffs: list) -> None:
        """coeffs : [a0, a1, a2, ...] represents
        P(x) = a0 + a1*x + a2*x^2 + ...
        """
        self._coeffs = coeffs
        
    def __call__(self, x):
        p = x * 0
        for coeff in self._coeffs[::-1]:
            p = x * p + coeff
        return p

In [4]:
class DualTest:
    def __init__(self, number_test_cases: int = 10) -> None:
        self._lhs = []
        self._rhs = []
        self.epsilon = 1e-10
        
        for _ in range(number_test_cases):
            self._lhs.append(Dual(rd.rand(), rd.rand()))
            self._rhs.append(Dual(rd.rand(), rd.rand()))
    
    def test_add(self) -> None:
        for A,B in zip(self._lhs, self._rhs):
            C = A+B #overloaded
            assert C._x == A._x + B._x
            assert C._y == A._y + B._y
    
    def test_sub(self) -> None:
        for A,B in zip(self._lhs, self._rhs):
            C = A-B #overloaded
            assert C._x == A._x - B._x
            assert C._y == A._y - B._y
    
    def test_mul(self) -> None:
        for A,B in zip(self._lhs, self._rhs):
            C = A * B #overloaded
            assert C._x == A._x * B._x
            assert C._y == A._x * B._y + A._y * B._x
    
    def test_pow(self) -> None:
        #Only test real number exponentiation and that (0,1) ** 2 == (0,0)
        for A in self._lhs:
            for k in [0, 1 , 2, 4, 10]:
                Ak = Dual(A._x, 0) ** k #overloaded
                assert np.abs( Ak._x - A._x ** k ) < self.epsilon
        assert Dual(0, 1) ** 2 == Dual(0, 0)

    def test_polynomial(self) -> None:
        #Test that P((x,1)) = (P(x),P'(x)) 
        for A in self._lhs:
            P1 = Polynomial([1])
            P2 = Polynomial([1, 1])
            P3 = Polynomial([1, 1, 1])
            B = Dual(A._x, 1)

            assert (P1(B) - Dual(P1(B._x),0)).norm() < self.epsilon 
            assert (P2(B) - Dual(P2(B._x),1)).norm() < self.epsilon 
            assert (P3(B) - Dual(P3(B._x),1+2*B._x)).norm() < self.epsilon
        
    def test_all(self) -> None:
        [getattr(self, func)() for func in dir(self) if callable(getattr(self, func)) 
                 and func.startswith("test") 
                 and not func == "test_all" ] 

In [6]:
Test = DualTest()
Test.test_all()